<a href="https://colab.research.google.com/github/wksmirnowa/machinelearning_homeworks/blob/master/HW04_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Кластеризация

## Импорты и подготовка

In [0]:
# !jupyter nbconvert --to html '/content/drive/My Drive/Colab Notebooks/HW04_clustering.ipynb'

[NbConvertApp] Converting notebook /content/drive/My Drive/Colab Notebooks/HW04_clustering.ipynb to html
[NbConvertApp] Writing 363662 bytes to /content/drive/My Drive/Colab Notebooks/HW04_clustering.html


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
from sklearn.cluster import AffinityPropagation, \
                            KMeans, MiniBatchKMeans, Birch, MeanShift, DBSCAN
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, \
                            silhouette_score

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM
from sklearn import metrics
from sklearn.datasets import make_blobs
import warnings
warnings.filterwarnings("ignore")

In [0]:
def show_metrics(X, y, labels):
  print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels)) # проверяет, что в кластере объекты одного класса
  print("Completeness: %0.3f" % metrics.completeness_score(y, labels)) # проверяет, что объекты класса только в одном кластере
  print("V-measure: %0.3f" % metrics.v_measure_score(y, labels)) # превращает обе метрики в одну 
  print("Adjusted Rand Index: %0.3f" % metrics.adjusted_rand_score(y, labels))
  print("Adjusted Mutual Information: %0.3f" % metrics.adjusted_mutual_info_score(y, labels))
  print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(X, labels))

In [0]:
def clasterisation(X, cluster_model):
  cluster_model.fit(X)
  labels = cluster_model.labels_
  return labels

In [0]:
data = pd.read_csv('/content/drive/My Drive/data.csv')

In [0]:
data.head(5)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1
0,793dd3d5f0f0,9467e0341c0e,Алтайский край,Барнаул,Личные вещи,Детская одежда и обувь,Для девочек,Платья и юбки,146-152 см (10-12 лет),Платье праздничное для девочки,Очень красивое торг,1500.0,106,2017-04-12,Company,ace7b2b7f066a6b6ff244f90d030c2c1a1bf118dd0e1ac...,573.0
1,f8c965d9b668,1905516660f5,Свердловская область,Каменск-Уральский,Для дома и дачи,Мебель и интерьер,"Кровати, диваны и кресла",NaN,NaN,Кровать двухспальная,Двуспальная кровать 140х200 с металлическим ка...,5700.0,10,2017-04-15,Private,b38ef24dd5b876ed626bd98241e228e4e19e4160d33abe...,1439.0
2,2f153c00d561,b4dc4849fdd4,Самарская область,Самара,Бытовая электроника,Телефоны,Samsung,NaN,NaN,Samsung S3 mini gt-i8190,"Продам телефон самсунг 2 ядра, 1 гиг оперативн...",1500.0,6,2017-04-12,Private,8530cc0c8d5dc6d212497dffd610d89b92a7db952934db...,2932.0
3,7fc5eea56915,02c14d39fdb4,Саратовская область,Саратов,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,44–46 (M),"Красивое платье на одно плечо, в идеале размер м","Заказывала в Сша, прекрасное качество, в деле ...",1000.0,1280,2017-04-13,Company,eb21745ba276d3b00b05856a58fe719e31a82419c0221a...,386.0
4,b01000d29ea3,b17f14110367,Удмуртия,Ижевск,Личные вещи,Детская одежда и обувь,Для мальчиков,"Шапки, варежки, шарфы",NaN,Стильная панама,Состояние новой. Размер до года- полтора. По ...,150.0,145,2017-04-14,Company,5bf1001fa681b96f121ad8047c85e3747f3dac37830dd0...,209.0


In [0]:
data = data[['category_name', 'title']]

In [0]:
data.shape

(254219, 2)

In [0]:
pd.set_option('display.max_rows', 500)

## Задание 1.

### AffinityPropagation

In [0]:
sample = data.sample(frac=0.01)

In [0]:
sample.shape

(2542, 2)

In [0]:
cv = CountVectorizer(max_df=300, min_df=2, max_features=500, ngram_range=(1,2))
tfidf = TfidfVectorizer(max_df=300, min_df=2, max_features=500, ngram_range=(1,2))
X_cv = cv.fit_transform(sample['title'])
X_tfidf = tfidf.fit_transform(sample['title'])

In [0]:
y = sample['category_name']

In [0]:
cluster_AP = AffinityPropagation(damping=0.9, preference=-10, 
                              max_iter=400, verbose=2)

In [0]:
%%time
labels_AP_cv = clasterisation(X=X_cv, cluster_model=cluster_AP)

Converged after 278 iterations.
CPU times: user 37.4 s, sys: 17.4 ms, total: 37.4 s
Wall time: 37.4 s


In [0]:
show_metrics(X=X_cv, y=y, labels=labels_AP_cv)

Homogeneity: 0.322
Completeness: 0.371
V-measure: 0.344
Adjusted Rand Index: -0.022
Adjusted Mutual Information: 0.246
Silhouette Coefficient: 0.280


In [0]:
%%time
labels_AP_tfidf = clasterisation(X=X_tfidf, cluster_model=cluster_AP)

Converged after 31 iterations.
CPU times: user 4.73 s, sys: 1.97 ms, total: 4.73 s
Wall time: 4.74 s


In [0]:
show_metrics(X=X_tfidf, y=y, labels=labels_AP_tfidf)

Homogeneity: 0.651
Completeness: 0.329
V-measure: 0.437
Adjusted Rand Index: 0.022
Adjusted Mutual Information: 0.124
Silhouette Coefficient: -0.288


In [0]:
len(set(labels_AP_cv))

75

In [0]:
len(set(labels_AP_tfidf))

585

In [0]:
sample['cluster_cv'] = labels_AP_cv

In [0]:
sample['cluster_tfidf'] = labels_AP_tfidf

У TfidfVectorizer выше Homogeneity, а у CountVectorizer – коэффициент силуэтов и Completeness. Это интересно потому, что какие-то метрики лучше у одного векторайзера, какие-то – у другого, то есть победителя по метрикам выбрать трудно. Но с точки зрения человеческого взгляда на раскластеризованные данные CountVectorizer, то есть векторайзер с более высоким коэффициентом силуэтов, выигрывает, потому что в целом лучше выделяет кластеры (чуть больше в целом похожих объявлений попадаются в одном кластере).

In [0]:
sample[sample.cluster_cv==50].head(10)

,category_name,title,cluster_cv,cluster_tfidf
134365,"Одежда, обувь, аксессуары",Новая сумочка GoldFish KP-02,50,21
223572,Товары для детей и игрушки,Коляска 2в1 индиана лен. Новая. коричневый,50,436
215729,"Одежда, обувь, аксессуары",Новая водолазка,50,119
128927,"Одежда, обувь, аксессуары","Новая юбка, об 102 см",50,192
56493,Красота и здоровье,Трость как новая,50,247
100405,Мебель и интерьер,Новая люстра,50,163
215354,Детская одежда и обувь,Стильная новая парка.98-110,50,491
5468,Мебель и интерьер,Кухня угловая новая с выставки,50,300
84386,Охота и рыбалка,Коптильня заводская почти новая.Пр-во Италия,50,377
181550,Бытовая техника,"Новая, ну или почти новая",50,379


In [0]:
sample[sample.cluster_tfidf==50].head(10)

,category_name,title,cluster_cv,cluster_tfidf
248337,Коммерческая недвижимость,Сниму помещение свободного назначения,53,50
190165,Предложение услуг,Манипулятор вездеход,0,50
204605,Детская одежда и обувь,Зимний комбенизон,0,50
230993,Оборудование для бизнеса,Алкотестре,0,50
236930,"Игры, приставки и программы",X box 360 обмен,0,50
3621,"Одежда, обувь, аксессуары",Джинсы женские большой размер 44,3,50
49339,"Одежда, обувь, аксессуары","Выходные босоножки, с гипюровыми ставками и ше...",74,50
208422,Автомобили,"ГАЗ ГАЗель, 2005",0,50


### KMeans

In [0]:
cv = CountVectorizer(max_df=300, min_df=2, max_features=500, ngram_range=(2,2))
tfidf = TfidfVectorizer(max_df=300, min_df=2, max_features=500, ngram_range=(2,2))
X_cv = cv.fit_transform(sample['title'])
X_tfidf = tfidf.fit_transform(sample['title'])

In [0]:
cluster_KMeans = KMeans(n_clusters=47, random_state=0, max_iter=1000, n_jobs=5, algorithm='full')

In [0]:
%%time
labels_KMeans_cv = clasterisation(X=X_cv, cluster_model=cluster_KMeans)

CPU times: user 52.3 ms, sys: 55.1 ms, total: 107 ms
Wall time: 2.91 s


In [0]:
%%time
labels_KMeans_tfidf = clasterisation(X=X_tfidf, cluster_model=cluster_KMeans)

CPU times: user 28.2 ms, sys: 4.06 ms, total: 32.3 ms
Wall time: 832 ms


In [0]:
show_metrics(X=X_cv, y=y, labels=labels_KMeans_cv)

Homogeneity: 0.144
Completeness: 0.535
V-measure: 0.227
Adjusted Rand Index: 0.016
Adjusted Mutual Information: 0.160
Silhouette Coefficient: 0.580


In [0]:
show_metrics(X=X_tfidf, y=y, labels=labels_KMeans_tfidf)

Homogeneity: 0.156
Completeness: 0.506
V-measure: 0.239
Adjusted Rand Index: 0.012
Adjusted Mutual Information: 0.168
Silhouette Coefficient: 0.623


In [0]:
len(set(labels_KMeans_cv))

47

In [0]:
len(set(labels_KMeans_tfidf))

47

In [0]:
sample['cluster_cv'] = labels_KMeans_cv

In [0]:
sample['cluster_tfidf'] = labels_KMeans_tfidf

Высокий коэффициет силуэтов и человеческий взгляд на данные показывают, что кластеризация прошла достаточно успешно и у того, и у другого векторайзера – много похожих объявлений в одном кластере. Они также сработали очень похоже. Победителем в этом раунде выбираю TfidfVectorizer, потому что у него лучше силуэты и Homogeneity.

In [0]:
sample[sample.cluster_cv==13].head(10)

,category_name,title,cluster_cv,cluster_tfidf
141619,Детская одежда и обувь,Ботинки для девочки Kapika,13,5
234179,Детская одежда и обувь,Плащ для девочки,13,5
173580,Детская одежда и обувь,"Жилет утепленный monna rosa, для девочки, 100 мик",13,5
105098,Детская одежда и обувь,Сапоги adidas для девочки,13,5
231506,Детская одежда и обувь,Туфли для девочки,13,5
149907,Детская одежда и обувь,To be Too песочник для девочки,13,5
198442,Детская одежда и обувь,Платье нарядное для девочки (на выпускной в са...,13,5
234307,Детская одежда и обувь,Брючки для девочки,13,5
153356,Детская одежда и обувь,Кроссовки для девочки,13,5
34503,Велосипеды,Велосипед для девочки,13,5


In [0]:
sample[sample.cluster_tfidf==5].head(10)

,category_name,title,cluster_cv,cluster_tfidf
141619,Детская одежда и обувь,Ботинки для девочки Kapika,13,5
234179,Детская одежда и обувь,Плащ для девочки,13,5
173580,Детская одежда и обувь,"Жилет утепленный monna rosa, для девочки, 100 мик",13,5
105098,Детская одежда и обувь,Сапоги adidas для девочки,13,5
231506,Детская одежда и обувь,Туфли для девочки,13,5
149907,Детская одежда и обувь,To be Too песочник для девочки,13,5
198442,Детская одежда и обувь,Платье нарядное для девочки (на выпускной в са...,13,5
234307,Детская одежда и обувь,Брючки для девочки,13,5
153356,Детская одежда и обувь,Кроссовки для девочки,13,5
34503,Велосипеды,Велосипед для девочки,13,5


### MiniBatchKMeans

In [0]:
cv = CountVectorizer(max_df=400, min_df=2, max_features=500, ngram_range=(1,2))
tfidf = TfidfVectorizer(max_df=400, min_df=2, max_features=500, ngram_range=(1,2))
X_cv = cv.fit_transform(sample['title'])
X_tfidf = tfidf.fit_transform(sample['title'])

In [0]:
cluster_MBKMeans = MiniBatchKMeans(n_clusters=1000, init_size=5000, \
                                   verbose=2, max_iter=1000, batch_size=1000, random_state=42, \
                                   max_no_improvement=200, reassignment_ratio=0.3)

In [132]:
%%time
labels_MBKMeans_cv = clasterisation(X=X_cv, cluster_model=cluster_MBKMeans)

Init 1/3 with method: k-means++
Inertia for init 1/3: 76.980854
Init 2/3 with method: k-means++
Inertia for init 2/3: 75.820979
Init 3/3 with method: k-means++
Inertia for init 3/3: 84.236766
Minibatch iteration 1/3000: mean batch inertia: 0.043279, ewa inertia: 0.043279 
Minibatch iteration 2/3000: mean batch inertia: 0.043528, ewa inertia: 0.043475 
Minibatch iteration 3/3000: mean batch inertia: 0.039146, ewa inertia: 0.040070 
Minibatch iteration 4/3000: mean batch inertia: 0.046436, ewa inertia: 0.045076 
Minibatch iteration 5/3000: mean batch inertia: 0.036471, ewa inertia: 0.038309 
Minibatch iteration 6/3000: mean batch inertia: 0.035039, ewa inertia: 0.035737 
Minibatch iteration 7/3000: mean batch inertia: 0.042755, ewa inertia: 0.041256 
Minibatch iteration 8/3000: mean batch inertia: 0.032278, ewa inertia: 0.034195 
Minibatch iteration 9/3000: mean batch inertia: 0.047878, ewa inertia: 0.044956 
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch iteration 10/3000:

In [133]:
%%time
labels_MBKMeans_tfidf = clasterisation(X=X_tfidf, cluster_model=cluster_MBKMeans)

Init 1/3 with method: k-means++
Inertia for init 1/3: 13.471714
Init 2/3 with method: k-means++
Inertia for init 2/3: 14.997625
Init 3/3 with method: k-means++
Inertia for init 3/3: 14.501479
Minibatch iteration 1/3000: mean batch inertia: 0.012609, ewa inertia: 0.012609 
Minibatch iteration 2/3000: mean batch inertia: 0.010126, ewa inertia: 0.010657 
Minibatch iteration 3/3000: mean batch inertia: 0.009550, ewa inertia: 0.009786 
Minibatch iteration 4/3000: mean batch inertia: 0.009082, ewa inertia: 0.009232 
Minibatch iteration 5/3000: mean batch inertia: 0.008572, ewa inertia: 0.008713 
Minibatch iteration 6/3000: mean batch inertia: 0.009336, ewa inertia: 0.009203 
Minibatch iteration 7/3000: mean batch inertia: 0.007931, ewa inertia: 0.008203 
Minibatch iteration 8/3000: mean batch inertia: 0.008459, ewa inertia: 0.008405 
Minibatch iteration 9/3000: mean batch inertia: 0.007702, ewa inertia: 0.007852 
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch iteration 10/3000:

In [134]:
show_metrics(X=X_cv, y=y, labels=labels_MBKMeans_cv)

Homogeneity: 0.657
Completeness: 0.384
V-measure: 0.484
Adjusted Rand Index: -0.004
Adjusted Mutual Information: 0.206
Silhouette Coefficient: 0.604


In [135]:
show_metrics(X=X_tfidf, y=y, labels=labels_MBKMeans_tfidf)

Homogeneity: 0.659
Completeness: 0.387
V-measure: 0.488
Adjusted Rand Index: -0.004
Adjusted Mutual Information: 0.217
Silhouette Coefficient: 0.625


In [136]:
len(set(labels_MBKMeans_cv))

666

In [137]:
len(set(labels_MBKMeans_cv))

666

In [0]:
sample['cluster_cv'] = labels_MBKMeans_cv

In [0]:
sample['cluster_tfidf'] = labels_MBKMeans_tfidf

In [140]:
sample[sample.cluster_cv==10].head(10)

,category_name,title,cluster_cv,cluster_tfidf
193410,"Дома, дачи, коттеджи",Дом 120 м² на участке 10 сот.,10,139
38470,"Дома, дачи, коттеджи",Дом 154 м² на участке 11 сот.,10,139
53796,"Дома, дачи, коттеджи",Дом 56 м² на участке 30 сот.,10,139
198427,"Дома, дачи, коттеджи",Коттедж 145 м² на участке 15 сот.,10,498
151199,"Дома, дачи, коттеджи",Дом 100 м² на участке 12 сот.,10,139
38100,"Дома, дачи, коттеджи",Дом 35 м² на участке 15 сот.,10,139
79777,"Дома, дачи, коттеджи",Дом 35 м² на участке 62 сот.,10,139


In [141]:
sample[sample.cluster_tfidf==10].head(10)

,category_name,title,cluster_cv,cluster_tfidf
104912,Детская одежда и обувь,Туфли Eli,151,10
174785,"Одежда, обувь, аксессуары",Туфли чёрные,151,10
27565,Детская одежда и обувь,Туфли,151,10
144353,"Одежда, обувь, аксессуары",Туфли,151,10
145504,"Одежда, обувь, аксессуары",Туфли Tipe Etacchi,151,10
114149,"Одежда, обувь, аксессуары",Туфли,151,10
23708,"Одежда, обувь, аксессуары",Туфли carlo pazolini,151,10
229504,"Одежда, обувь, аксессуары",Туфли (Испания),151,10
35920,"Одежда, обувь, аксессуары",Туфли nouchka,151,10
145530,"Одежда, обувь, аксессуары",Туфли,151,10


По метрикам с небольшим отрывом побеждает TfidfVectorizer, хотя в целом оба векторайзера плюс-минус одинаково отработали.

Общий вывод такой: оба векторайзера хороши, TfidfVectorizer в общем побеждает, но с совсем небольшим отрывом по метрикам.

## Задание 2.

In [0]:
tfidf = TfidfVectorizer(max_df=300, min_df=2, max_features=500, ngram_range=(1,2))
X = tfidf.fit_transform(sample['title'])
svd = TruncatedSVD(50, random_state=0)
X_svd = svd.fit_transform(X_cv)
nmf = NMF(50, random_state=0)
X_nmf = nmf.fit_transform(X_cv)

In [160]:
%%time
labels_AP_svd = clasterisation(X=X_svd, cluster_model=cluster_AP)

Converged after 191 iterations.
CPU times: user 22.3 s, sys: 130 ms, total: 22.5 s
Wall time: 22.4 s


In [161]:
show_metrics(X=X_svd, y=y, labels=labels_AP_svd)

Homogeneity: 0.301
Completeness: 0.407
V-measure: 0.346
Adjusted Rand Index: -0.001
Adjusted Mutual Information: 0.272
Silhouette Coefficient: 0.673


In [162]:
%%time
labels_AP_nmf = clasterisation(X=X_nmf, cluster_model=cluster_AP)

Converged after 71 iterations.
CPU times: user 8.96 s, sys: 103 ms, total: 9.06 s
Wall time: 8.94 s


In [163]:
show_metrics(X=X_nmf, y=y, labels=labels_AP_nmf)

Homogeneity: 0.071
Completeness: 0.763
V-measure: 0.129
Adjusted Rand Index: 0.015
Adjusted Mutual Information: 0.117
Silhouette Coefficient: 0.422


C AffinityPropagation по нескольким метрикам побеждает SVD 

In [164]:
%%time
labels_KMeans_svd = clasterisation(X=X_svd, cluster_model=cluster_KMeans)

CPU times: user 35.3 ms, sys: 5.05 ms, total: 40.4 ms
Wall time: 391 ms


In [165]:
show_metrics(X=X_svd, y=y, labels=labels_KMeans_svd)

Homogeneity: 0.291
Completeness: 0.396
V-measure: 0.335
Adjusted Rand Index: -0.004
Adjusted Mutual Information: 0.260
Silhouette Coefficient: 0.669


In [166]:
%%time
labels_KMeans_nmf = clasterisation(X=X_nmf, cluster_model=cluster_KMeans)

CPU times: user 40.6 ms, sys: 1.27 ms, total: 41.8 ms
Wall time: 388 ms


In [167]:
show_metrics(X=X_nmf, y=y, labels=labels_KMeans_nmf)

Homogeneity: 0.295
Completeness: 0.391
V-measure: 0.336
Adjusted Rand Index: -0.006
Adjusted Mutual Information: 0.259
Silhouette Coefficient: 0.696


C KMeans по нескольким метрикам с совсем маленьким отрывом побеждает NMF 

In [168]:
%%time
labels_MBKMeans_svd = clasterisation(X=X_svd, cluster_model=cluster_MBKMeans)

Init 1/3 with method: k-means++
Inertia for init 1/3: 0.012165
Init 2/3 with method: k-means++
Inertia for init 2/3: 0.010730
Init 3/3 with method: k-means++
Inertia for init 3/3: 0.013154
Minibatch iteration 1/3000: mean batch inertia: 0.000008, ewa inertia: 0.000008 
Minibatch iteration 2/3000: mean batch inertia: 0.000006, ewa inertia: 0.000007 
Minibatch iteration 3/3000: mean batch inertia: 0.000007, ewa inertia: 0.000007 
Minibatch iteration 4/3000: mean batch inertia: 0.000006, ewa inertia: 0.000006 
Minibatch iteration 5/3000: mean batch inertia: 0.000006, ewa inertia: 0.000006 
Minibatch iteration 6/3000: mean batch inertia: 0.000006, ewa inertia: 0.000006 
Minibatch iteration 7/3000: mean batch inertia: 0.000006, ewa inertia: 0.000006 
Minibatch iteration 8/3000: mean batch inertia: 0.000004, ewa inertia: 0.000005 
Minibatch iteration 9/3000: mean batch inertia: 0.000008, ewa inertia: 0.000007 
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch iteration 10/3000: me

In [169]:
show_metrics(X=X_svd, y=y, labels=labels_MBKMeans_svd)

Homogeneity: 0.654
Completeness: 0.379
V-measure: 0.480
Adjusted Rand Index: -0.004
Adjusted Mutual Information: 0.197
Silhouette Coefficient: 0.599


In [170]:
%%time
labels_MBKMeans_nmf = clasterisation(X=X_nmf, cluster_model=cluster_MBKMeans)

Init 1/3 with method: k-means++
Inertia for init 1/3: 0.000145
Init 2/3 with method: k-means++
Inertia for init 2/3: 0.000135
Init 3/3 with method: k-means++
Inertia for init 3/3: 0.000145
Minibatch iteration 1/3000: mean batch inertia: 0.000000, ewa inertia: 0.000000 
Minibatch iteration 2/3000: mean batch inertia: 0.000000, ewa inertia: 0.000000 
Minibatch iteration 3/3000: mean batch inertia: 0.000000, ewa inertia: 0.000000 
Minibatch iteration 4/3000: mean batch inertia: 0.000000, ewa inertia: 0.000000 
Minibatch iteration 5/3000: mean batch inertia: 0.000000, ewa inertia: 0.000000 
Minibatch iteration 6/3000: mean batch inertia: 0.000000, ewa inertia: 0.000000 
Minibatch iteration 7/3000: mean batch inertia: 0.000000, ewa inertia: 0.000000 
Minibatch iteration 8/3000: mean batch inertia: 0.000000, ewa inertia: 0.000000 
Minibatch iteration 9/3000: mean batch inertia: 0.000000, ewa inertia: 0.000000 
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch iteration 10/3000: me

In [171]:
show_metrics(X=X_nmf, y=y, labels=labels_KMeans_nmf)

Homogeneity: 0.295
Completeness: 0.391
V-measure: 0.336
Adjusted Rand Index: -0.006
Adjusted Mutual Information: 0.259
Silhouette Coefficient: 0.696


C MiniBatchKMeans по Homogeneity побеждает SVD, но по силуэтам – NMF. Учитывая, что отрыв по силуэтам у разложений небольшой, полным победителем в этом раунде назову SVD.

Итак, в 2 из 3 алгоритмов победил SVD.

## Задание 3.

Чтобы выявить выбросы, я решила обратиться к готовым алгоритмам из sklearn – EllipticEnvelope и OneClassSVM, про которые прочитала в [этой статье](https://dyakonov.org/2017/04/19/поиск-аномалий-anomaly-detection/). Правда, не могу сказать, что то, что я получила, действительно похоже на выбросы. Возможно, дело в том, что данные и так разнообразные, поэтому выявить настоящие выбросы проблематично. Во всяком случае интересно, что у обоих алгоритмов есть пересечения по аномальности, но не везде, и OneClassSVM разметил, как выбросы, больше данных, чем EllipticEnvelope. Я также попробовала IsolationForest, но он совсем не справился со своей задачей (не нашел никаких выбросов), поэтому его тут нет.

In [0]:
X_tfidf = tfidf.fit_transform(sample['title'])

In [0]:
ee = EllipticEnvelope()
sample['anomaly'] = ee.fit_predict(X_tfidf.toarray())

In [89]:
sample[sample.anomaly==-1].sample(20)

,category_name,title,anomaly,anomaly_svm
206583,Товары для детей и игрушки,Игровой туннель Дракоша,-1,1
53459,Детская одежда и обувь,Ботиночки,-1,1
60634,"Одежда, обувь, аксессуары",Брюки и джинсы (Германия) art.12apr/44,-1,-1
180184,Детская одежда и обувь,Костюмчик демисезонний,-1,-1
229488,"Одежда, обувь, аксессуары",Зимние оранжевые ботинки,-1,-1
192012,"Дома, дачи, коттеджи",Дом 45 м² на участке 7 сот.,-1,-1
100946,Автомобили,"ГАЗ ГАЗель 3302, 1997",-1,-1
19083,"Одежда, обувь, аксессуары",Летняя обувь,-1,1
35437,Бытовая техника,Холодильник,-1,-1
184213,Аудио и видео,Panasonic NV FS 100,-1,1


In [87]:
sample[sample.anomaly==-1].category_name.value_counts()

Детская одежда и обувь         64
Одежда, обувь, аксессуары      58
Дома, дачи, коттеджи           41
Предложение услуг              16
Квартиры                       14
Автомобили                      8
Ремонт и строительство          7
Товары для детей и игрушки      7
Бытовая техника                 6
Товары для компьютера           5
Телефоны                        4
Гаражи и машиноместа            3
Мебель и интерьер               2
Спорт и отдых                   2
Красота и здоровье              2
Ноутбуки                        2
Велосипеды                      2
Игры, приставки и программы     2
Охота и рыбалка                 1
Комнаты                         1
Оргтехника и расходники         1
Книги и журналы                 1
Коллекционирование              1
Мотоциклы и мототехника         1
Фототехника                     1
Аудио и видео                   1
Коммерческая недвижимость       1
Музыкальные инструменты         1
Name: category_name, dtype: int64

In [88]:
len(sample[sample.anomaly==-1])

255

In [0]:
svm = OneClassSVM()
sample['anomaly_svm'] = svm.fit_predict(X_tfidf)

In [91]:
sample[sample.anomaly_svm==-1].sample(20)

,category_name,title,anomaly,anomaly_svm
160802,Автомобили,"Ford Focus, 2006",1,-1
71356,Детская одежда и обувь,Кепка розовая,1,-1
153398,Телефоны,Смартфон Digma Linx C500 3G Новый,1,-1
192082,"Одежда, обувь, аксессуары",Кроссовки; кеды adidas superstar originals,1,-1
158461,Детская одежда и обувь,"Купальник для танцев, гимнастики белый",1,-1
245628,"Одежда, обувь, аксессуары",Плащ,1,-1
165878,Товары для детей и игрушки,Детская коляска 3 в 1,-1,-1
142506,Квартиры,"1-к квартира, 32 м², 3/6 эт.",1,-1
152754,Автомобили,"LADA Priora, 2008",1,-1
204910,Фототехника,Фотоэкспонометр свердловск 4 (СССР),1,-1


In [92]:
sample[sample.anomaly_svm==-1].category_name.value_counts()

Детская одежда и обувь          237
Одежда, обувь, аксессуары       225
Товары для детей и игрушки       99
Автомобили                       95
Телефоны                         78
Квартиры                         69
Мебель и интерьер                58
Ремонт и строительство           40
Предложение услуг                38
Товары для компьютера            36
Бытовая техника                  30
Велосипеды                       23
Аудио и видео                    23
Земельные участки                23
Дома, дачи, коттеджи             22
Коллекционирование               19
Ноутбуки                         15
Спорт и отдых                    14
Игры, приставки и программы      11
Комнаты                          10
Гаражи и машиноместа             10
Красота и здоровье               10
Фототехника                       9
Товары для животных               9
Планшеты и электронные книги      9
Грузовики и спецтехника           9
Книги и журналы                   9
Настольные компьютеры       

In [93]:
len(sample[sample.anomaly_svm==-1])

1303

Попробуем использовать еще MeanShift, который сам определяет выбросы. Ему как-то верится еще больше, поскольку в качестве выбросов он показывает объявления, в которых заголовок просто повторяет название категории или имеет неожиданные слова, ошибки и т.д.

In [35]:
import sklearn
sklearn.cluster.estimate_bandwidth(X_svd)

0.7380776573061245

In [36]:
cluster = MeanShift(cluster_all=False, bandwidth=0.7)
cluster.fit(X_svd)

MeanShift(bandwidth=0.7, bin_seeding=False, cluster_all=False, max_iter=300,
          min_bin_freq=1, n_jobs=None, seeds=None)

In [0]:
labels = cluster.labels_

In [0]:
sample['cluster_MS'] = labels

In [96]:
sample[sample.cluster_MS==-1].sample(20)

,category_name,title,anomaly,anomaly_svm,cluster_MS
63211,Телефоны,Смартфон ZTE Blade A610 Арт.16992,1,-1,-1
185628,Детская одежда и обувь,Детские вещи,1,1,-1
205752,Детская одежда и обувь,Детские вещи,1,1,-1
184381,Телефоны,Смартфон Apple iPhone 6 Plus 16GB Арт. 39936,1,-1,-1
42517,Телефоны,Смартфон iPhone 6S 16Gb арт.33261,1,1,-1
173808,Детская одежда и обувь,Продаю одежду пакетом,1,-1,-1
214221,"Дома, дачи, коттеджи",Дом 33 м² на участке 17 сот.,-1,-1,-1
157288,Мебель и интерьер,Тумба под тв,1,-1,-1
17195,Ремонт и строительство,Натяжной потолок арт 65,1,-1,-1
32207,Коллекционирование,Коробочка ркка под зубной порошок,1,-1,-1


In [97]:
sample[sample.cluster_MS==-1].category_name.value_counts()

Детская одежда и обувь        9
Телефоны                      7
Ремонт и строительство        4
Одежда, обувь, аксессуары     4
Товары для детей и игрушки    3
Гаражи и машиноместа          2
Книги и журналы               2
Дома, дачи, коттеджи          2
Ноутбуки                      1
Товары для компьютера         1
Коллекционирование            1
Предложение услуг             1
Мебель и интерьер             1
Бытовая техника               1
Автомобили                    1
Оборудование для бизнеса      1
Посуда и товары для кухни     1
Name: category_name, dtype: int64

In [98]:
len(sample[sample.cluster_MS==-1])

42

Еще попробуем DBSCAN.

In [116]:
cluster = DBSCAN(min_samples=10, eps=0.5) 
cluster.fit(X_svd)

DBSCAN(algorithm='auto', eps=0.5, leaf_size=30, metric='euclidean',
       metric_params=None, min_samples=10, n_jobs=None, p=None)

In [0]:
labels = cluster.labels_

In [0]:
sample['cluster_DBSCAN'] = labels

Его выбросы тоже так себе – не совсем похоже на выбросы.

In [119]:
sample[sample.cluster_DBSCAN==-1].sample(20)

,category_name,title,anomaly,anomaly_svm,cluster_MS,cluster_DBSCAN
228833,Квартиры,"1-к квартира, 38 м², 10/17 эт.",1,-1,108,-1
216683,Детская одежда и обувь,Продам ботинки,1,1,74,-1
70161,Квартиры,"2-к квартира, 45 м², 1/5 эт.",1,1,38,-1
45579,Товары для детей и игрушки,Детский стульчик для кормления,1,1,41,-1
3983,Квартиры,"1-к квартира, 33.7 м², 1/5 эт.",1,1,4,-1
29057,"Дома, дачи, коттеджи",Дом 65 м² на участке 25 сот.,1,1,69,-1
33258,Комнаты,"Комната 60 м² в 3-к, 5/9 эт.",1,-1,119,-1
8145,Квартиры,"2-к квартира, 50 м², 5/5 эт.",1,-1,45,-1
38143,Квартиры,"2-к квартира, 45.5 м², 3/5 эт.",1,1,38,-1
235872,Детская одежда и обувь,Куртка детская,-1,1,104,-1


In [120]:
len(sample[sample.cluster_DBSCAN==-1])

448

Итог: пока что лучше всех с поисками выбросов справился MeanShift, потому что у него видно больше всего объявлений, отличающихся от большинства. К тому же он нашел меньше всего выбросов, и мне кажется, что 42 выброса – намного более адекватное число, чем 1303 или даже 448.